# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 767 ms, sys: 263 ms, total: 1.03 s
Wall time: 588 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 5.48 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh1/periodic_180_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh1/periodic_180_6561"
random_state = 257


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1180980 entries, 0 to 1180979
Columns: 188 entries, id to feat179
dtypes: float64(183), int32(3), object(2)
memory usage: 1.6+ GB
CPU times: user 17 s, sys: 896 ms, total: 17.9 s
Wall time: 17.9 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.074536,0.074536,...,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.105409,0.105345,...,0.099052,0.097734,0.101326,0.100250,0.103106,0.102278,0.104383,0.103808,0.105152,0.104832
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.105409,0.105152,...,0.080748,0.075825,0.089392,0.085278,0.096296,0.093071,0.101326,0.099052,0.104383,0.103106
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,-0.074536,0.074536,...,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.000030,-0.003649,...,-0.036024,-0.039459,-0.029026,-0.032545,-0.021886,-0.025472,-0.014640,-0.018274,-0.007323,-0.010988


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  180
n_hamiltonians:  6561
n_ts:  2
CPU times: user 184 µs, sys: 12 µs, total: 196 µs
Wall time: 99.9 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.9358329522938577
% val:  0.0
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  6140
number of val hamiltonians:  0
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:33<55:21, 33.55s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:06<54:32, 33.39s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [53:17<00:00, 31.69s/it]

running experiments: 100%|██████████| 100/100 [53:17<00:00, 31.98s/it]

CPU times: user 53min 10s, sys: 8.02 s, total: 53min 18s
Wall time: 53min 17s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.074536,0.074536,...,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.105409,0.105345,...,0.099052,0.097734,0.101326,0.100250,0.103106,0.102278,0.104383,0.103808,0.105152,0.104832
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.105409,0.105152,...,0.080748,0.075825,0.089392,0.085278,0.096296,0.093071,0.101326,0.099052,0.104383,0.103106
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.074536,0.074536,...,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.000030,-0.003649,...,-0.036024,-0.039459,-0.029026,-0.032545,-0.021886,-0.025472,-0.014640,-0.018274,-0.007323,-0.010988
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.000365,0.006989,...,0.067476,0.072960,0.055548,0.061662,0.042540,0.049164,0.028703,0.035709,0.014308,0.021558
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.105377,-0.104528,...,-0.050435,-0.040484,-0.068577,-0.059834,-0.083723,-0.076569,-0.095209,-0.089958,-0.102533,-0.099416
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.002187,-0.013191,...,-0.092361,-0.097165,-0.079781,-0.086545,-0.063714,-0.072143,-0.044863,-0.054587,-0.024050,-0.034646
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.105391,0.104094,...,0.016379,0.001727,0.044446,0.030711,0.069070,0.057316,0.088342,0.079480,0.100770,0.095486
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.000845,0.015506,...,0.103951,0.105371,0.095109,0.100508,0.078897,0.087858,0.056573,0.068401,0.029866,0.043644


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 169 ms, sys: 3 µs, total: 169 ms
Wall time: 168 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179
948336,5268,H_1.25_-1.85_1.00065.dat,1.25,-1.85,1.000647,1,1,val,0.105409,0.052495,...,0.052705,0.105409,0.096296,0.010778,-0.032573,-0.103156,-0.103106,-0.032343,0.011018,0.096394
1073909,5966,H_1.65_0.65_-0.000704987.dat,1.65,0.65,-0.000705,0,0,val,0.000059,0.006163,...,-0.091258,0.094212,-0.078295,0.082324,-0.061911,0.066838,-0.042820,0.048430,-0.021858,0.027906
376471,2091,H_-0.75_1.3_1.00069.dat,-0.75,1.30,1.000688,1,1,val,-0.074536,0.074536,...,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536,-0.074536,0.074536
302395,1679,H_-1_0.95_0.000528392.dat,-1.00,0.95,0.000528,0,0,val,-0.001237,-0.094495,...,0.068699,0.042362,-0.006118,0.097523,-0.077501,0.097942,-0.105381,0.043384,-0.074109,-0.035526
57888,321,H_-1.85_1.9_1.00055.dat,-1.85,1.90,1.000550,1,0,val,0.074536,0.074536,...,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536,0.074536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911098,5061,H_1.1_-0.05_4.71562e-05.dat,1.10,-0.05,0.000047,0,0,val,0.014162,0.009495,...,0.049034,0.044828,-0.078676,-0.081724,-0.087101,-0.084369,0.036533,0.040902,0.104777,0.104159
1151050,6394,H_1.9_1.8_-0.019536.dat,1.90,1.80,-0.019536,0,1,test,0.105409,0.045922,...,0.080748,0.096166,-0.105152,-0.039192,0.070533,-0.039782,0.003679,0.096425,-0.075825,-0.098943
289801,1610,H_-1.05_1.55_1.00065.dat,-1.05,1.55,1.000647,1,1,val,-0.074536,-0.074536,...,0.074536,0.074536,-0.074536,-0.074536,0.074536,0.074536,-0.074536,-0.074536,0.074536,0.074536
993410,5518,H_1.4_-1.5_1.00056.dat,1.40,-1.50,1.000561,1,1,train,0.105404,-0.004073,...,-0.051793,-0.089182,-0.009974,-0.104328,0.033570,-0.101434,0.071309,-0.081002,0.096718,-0.046563


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.09358329522938577
% val:  0.8422496570644719
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  614
number of val hamiltonians:  5526
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [2575, 5584, 5051, 5518, 6106, 1194, 3413, 2942, 3762, 2409, 99, 3708, 5635, 4471, 1245, 4863, 3292, 5900, 109, 680, 6490, 5675, 4295, 4029, 1119, 889, 4380, 1185, 4132, 4007, 3648, 322, 2490, 6308, 5313, 3049, 2854, 3284, 108, 5140, 5300, 4386, 3805, 2633, 4627, 2095, 2224, 3201, 4790, 6470, 4449, 1699, 2063, 3031, 1088, 1943, 3775, 6022, 2229, 947, 5316, 1574, 6103, 3048, 1910, 6245, 3128, 2523, 6273, 2704, 1879, 2493, 3821, 4112, 599, 954, 3414, 127, 915, 3596, 6136, 1192, 4298, 4047, 2994, 2680, 727, 6517, 6241, 5977, 4985, 3576, 1755, 4950, 5283, 4571, 190, 6529, 3625, 4714, 6526, 49, 3503, 6169, 6044, 4456, 478, 3709, 3173, 6198, 1504, 1258, 30, 2238, 3562, 2613, 4326, 2666, 6031, 6333, 6435, 2286, 5243, 6117, 5041, 

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 133 µs, sys: 0 ns, total: 133 µs
Wall time: 77.2 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.527778,0.472222,999,0
1,-2.0,-1.95,test,0.733333,0.266667,0,0
2,-2.0,-1.90,test,0.800000,0.200000,0,0
3,-2.0,-1.85,test,0.888889,0.111111,0,0
4,-2.0,-1.80,val,0.911111,0.088889,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,val,0.938889,0.061111,0,0
6557,2.0,1.85,test,0.883333,0.116667,0,0
6558,2.0,1.90,test,0.866667,0.133333,0,0


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1180980


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,1,test
2,0,999,0,test
3,0,999,1,test
4,0,999,1,test
...,...,...,...,...
1180975,6560,999,0,test
1180976,6560,999,0,test
1180977,6560,999,0,test
1180978,6560,999,0,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9895765472312703,
 'eigenvector_val': 0.9722202115253147,
 'eigenvector_test': 0.8300213675213676,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 1.0,
 'hamiltonian_test': 1.0}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}